In [15]:
pip install SQLAlchemy


Note: you may need to restart the kernel to use updated packages.


In [16]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, DeclarativeBase, Mapped, mapped_column, relationship
from typing import List, Optional


# Define the database URL (using SQLite for this example)
DATABASE_URL = "sqlite:///./test.db"

# Create the database engine
engine = create_engine(DATABASE_URL, echo=True)

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(
        back_populates="user", cascade="all, delete-orphan"
    )
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    user: Mapped["User"] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [17]:
from sqlalchemy import create_engine
engine = create_engine("sqlite://", echo=True)

In [18]:
Base.metadata.create_all(engine)

2024-09-09 19:34:59,579 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-09 19:34:59,579 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-09-09 19:34:59,579 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-09 19:34:59,579 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-09-09 19:34:59,579 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-09 19:34:59,579 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-09-09 19:34:59,579 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-09 19:34:59,579 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-09-09 19:34:59,579 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-09 19:34:59,579 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2024-09-09 19:34:59,579 INFO sqlalchemy.engine.Engine [no key 0.00060s] ()
2024-09-09 19:34:59,579 INFO sqlalchemy.engine.

In [19]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )
    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )
    patrick = User(name="patrick", fullname="Patrick Star")
    session.add_all([spongebob, sandy, patrick])
    session.commit()

2024-09-09 19:34:59,610 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-09 19:34:59,610 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-09-09 19:34:59,610 INFO sqlalchemy.engine.Engine [generated in 0.00017s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('spongebob', 'Spongebob Squarepants')
2024-09-09 19:34:59,610 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-09-09 19:34:59,610 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('sandy', 'Sandy Cheeks')
2024-09-09 19:34:59,610 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-09-09 19:34:59,610 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] ('patrick', 'Patrick Star')
2024-09-09 19:34:59,626 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?) RETURN

In [20]:
from sqlalchemy import select

session = Session(engine)

stmt = select(User).where(User.name.in_(["spongebob", "sandy"]))

for user in session.scalars(stmt):
    print(user)

2024-09-09 19:34:59,642 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-09 19:34:59,642 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name IN (?, ?)
2024-09-09 19:34:59,642 INFO sqlalchemy.engine.Engine [generated in 0.00122s] ('spongebob', 'sandy')
User(id=1, name='spongebob', fullname='Spongebob Squarepants')
User(id=2, name='sandy', fullname='Sandy Cheeks')


In [21]:
# To query against multiple tables
stmt = (
    select(Address)
    .join(Address.user)
    .where(User.name == "sandy")
    .where(Address.email_address == "sandy@sqlalchemy.org")
)
sandy_address = session.scalars(stmt).one()
sandy_address

2024-09-09 19:34:59,657 INFO sqlalchemy.engine.Engine SELECT address.id, address.email_address, address.user_id 
FROM address JOIN user_account ON user_account.id = address.user_id 
WHERE user_account.name = ? AND address.email_address = ?
2024-09-09 19:34:59,657 INFO sqlalchemy.engine.Engine [generated in 0.00076s] ('sandy', 'sandy@sqlalchemy.org')


Address(id=2, email_address='sandy@sqlalchemy.org')

In [22]:
# Making changes
stmt = select(User).where(User.name == "patrick")
patrick = session.scalars(stmt).one()
patrick.addresses.append(Address(email_address="patrickstar@sqlalchemy.org"))
sandy_address.email_address = "sandy_cheeks@sqlalchemy.org"

session.commit()

2024-09-09 19:34:59,673 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-09-09 19:34:59,673 INFO sqlalchemy.engine.Engine [generated in 0.00100s] ('patrick',)
2024-09-09 19:34:59,673 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2024-09-09 19:34:59,673 INFO sqlalchemy.engine.Engine [generated in 0.00071s] (3,)
2024-09-09 19:34:59,673 INFO sqlalchemy.engine.Engine UPDATE address SET email_address=? WHERE address.id = ?
2024-09-09 19:34:59,673 INFO sqlalchemy.engine.Engine [generated in 0.00072s] ('sandy_cheeks@sqlalchemy.org', 2)
2024-09-09 19:34:59,673 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2024-09-09 19:34:59,673 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ('patrickstar@sqlalchemy.org', 3)


In [23]:
sandy = session.get(User, 2)
sandy.addresses.remove(sandy_address)

2024-09-09 19:34:59,704 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-09 19:34:59,704 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2024-09-09 19:34:59,704 INFO sqlalchemy.engine.Engine [generated in 0.00118s] (2,)
2024-09-09 19:34:59,704 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2024-09-09 19:34:59,704 INFO sqlalchemy.engine.Engine [cached since 0.03258s ago] (2,)


In [24]:
session.flush()

2024-09-09 19:35:32,493 INFO sqlalchemy.engine.Engine DELETE FROM address WHERE address.id = ?
2024-09-09 19:35:32,495 INFO sqlalchemy.engine.Engine [generated in 0.00159s] (2,)


In [25]:
session.delete(patrick)

2024-09-09 19:35:38,139 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2024-09-09 19:35:38,140 INFO sqlalchemy.engine.Engine [cached since 38.43s ago] (3,)
2024-09-09 19:35:38,141 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2024-09-09 19:35:38,142 INFO sqlalchemy.engine.Engine [cached since 38.46s ago] (3,)


In [26]:
session.delete(patrick)

In [27]:
session.query(User).all()

2024-09-09 19:35:56,067 INFO sqlalchemy.engine.Engine DELETE FROM address WHERE address.id = ?
2024-09-09 19:35:56,068 INFO sqlalchemy.engine.Engine [cached since 23.57s ago] (4,)
2024-09-09 19:35:56,069 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.id = ?
2024-09-09 19:35:56,070 INFO sqlalchemy.engine.Engine [generated in 0.00066s] (3,)
2024-09-09 19:35:56,071 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account
2024-09-09 19:35:56,071 INFO sqlalchemy.engine.Engine [generated in 0.00071s] ()


[User(id=1, name='spongebob', fullname='Spongebob Squarepants'),
 User(id=2, name='sandy', fullname='Sandy Cheeks')]

In [28]:
session.query(Address).all()

2024-09-09 19:36:12,928 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address
2024-09-09 19:36:12,929 INFO sqlalchemy.engine.Engine [generated in 0.00117s] ()


[Address(id=1, email_address='spongebob@sqlalchemy.org'),
 Address(id=3, email_address='sandy@squirrelpower.org')]

In [30]:
session.add(User(name='rohit', fullname='rohit sharma'))
session.commit()

2024-09-09 19:37:08,123 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-09-09 19:37:08,124 INFO sqlalchemy.engine.Engine [cached since 128.5s ago (insertmanyvalues) 1/2 (ordered; batch not supported)] ('rohit', 'rohit sharma')
2024-09-09 19:37:08,124 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-09-09 19:37:08,125 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('rohit', 'rohit sharma')
2024-09-09 19:37:08,128 INFO sqlalchemy.engine.Engine COMMIT


In [31]:
session.query(User).all()

2024-09-09 19:37:20,180 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-09 19:37:20,182 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account
2024-09-09 19:37:20,183 INFO sqlalchemy.engine.Engine [cached since 84.11s ago] ()


[User(id=1, name='spongebob', fullname='Spongebob Squarepants'),
 User(id=2, name='sandy', fullname='Sandy Cheeks'),
 User(id=3, name='rohit', fullname='rohit sharma'),
 User(id=4, name='rohit', fullname='rohit sharma')]

In [34]:
query = select(User).where(User.name=='rohit')
session.scalar(query).all()

2024-09-09 19:39:20,734 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-09-09 19:39:20,735 INFO sqlalchemy.engine.Engine [cached since 261.1s ago] ('rohit',)


AttributeError: 'User' object has no attribute 'all'